In [ ]:
# Mike Christ Heru - 2001572080 - LA05

# [15%] Write a simple python code (in a Python notebook please) implementing:
# an unsmoothed unigram, bigram, and trigram language model. 
# Your code should be able to estimate the probability of a given sentence and generate a new sentence


import math
import nltk
from nltk.tokenize import sent_tokenize as st
from nltk.tokenize import word_tokenize as wt

unigram_dict = {}
bigram_dict = {}
trigram_dict = {}
SENTENCE_START = "<s>"
SENTENCE_END = "</s>"
unigram_corpus_length = 0
bigram_corpus_length = 0
trigram_corpus_length = 0
unigram_unique_words = 0
bigram_unique_words = 0
trigram_unique_words = 0
bigram_unique_set = set()
trigram_unique_set = set()

def calcgram(s):
	return len(s) - 2

def calculate_unigram(s):
	res = 0
	for sentence in s:
		res += len(sentence) - 2
	return res

def calculate_bigram(s):
	res = 0
	for sentence in s:
		res += len(sentence) - 1
	return res

def calculate_trigram(s):
	res = 0
	for sentence in s:
		res += len(sentence) - 2
	return res

def preprocess_sentence(s):
  new_words = [SENTENCE_START]
  for word in wt(s):
      new_words.append(word)
  new_words.append(SENTENCE_END)
  return new_words

def preprocess_sentences(s):
  sentences = st(s)
  new_sentences = []
  for sentence in sentences:
    new_sentences.append(preprocess_sentence(sentence[:-1])[:])
  return new_sentences

def preprocess(s):
  unigram(s)
  bigram(s)
  trigram(s)

def unigram(s):
  global unigram_corpus_length
  for sentence in s:
    for word in sentence:
      try:
        unigram_dict[word] += 1
      except:
        unigram_dict[word] = 1
      if(word != SENTENCE_START and word != SENTENCE_END):
        unigram_corpus_length += 1
  unigram_unique_words = len(unigram_dict) - 2


def bigram(s):
  global bigram_corpus_length
  for sentence in s:
    previous_word = None
    for word in sentence:
      if(previous_word != None):
        try:
          bigram_dict[(previous_word,word)] += 1
        except:
          bigram_dict[(previous_word,word)] = 1
        if previous_word != SENTENCE_START and word != SENTENCE_END:
          bigram_unique_set.add((previous_word, word))
      previous_word = word
  bigram_unique_words = len(bigram_dict)

def trigram(s):
  global trigram_corpus_length
  for sentence in s:
    first_word = None
    second_word = None
    for word in sentence:
      if(first_word != None and second_word != None):
        try:
          trigram_dict[(first_word,second_word,word)] += 1
        except:
          trigram_dict[(first_word,second_word,word)] = 1
        if first_word != SENTENCE_START and word != SENTENCE_END:
          trigram_unique_set.add((first_word,second_word,word))
      first_word = second_word
      second_word = word
  trigram_unique_words = len(trigram_dict)

def probability_unigram(word,smooth):
	try:
		result_numerator = unigram_dict[word]
	except:
		result_numerator = 0
	try:
		result_denumerator = unigram_unique_words
	except:
		result_denumerator = 0
	if(smooth):
		result_numerator += 1
		result_denumerator += 1
	return float(result_numerator) / float(result_denumerator)

def probability_bigram(previous_word,word,smooth):
	try:
		result_numerator = bigram_dict[(previous_word,word)]
	except:
		result_numerator = 0
	try:
		result_denumerator = unigram_dict[previous_word]
	except:
		result_denumerator = 0
	if(smooth):
		result_numerator += 1
		result_denumerator += 1
	if(result_numerator == 0 or result_denumerator ==0):
		return 0.0
	return float(result_numerator) / float(result_denumerator)

def probability_trigram(first_word,second_word,word,smooth):
	try:
		result_numerator = trigram_dict[(first_word,second_word,word)]
	except:
		result_numerator = 0
	try:
		result_denumerator = bigram_dict[(first_word,second_word)]
	except:
		result_denumerator = 0
	if(smooth):
		result_numerator += 1
		result_denumerator += 1
	if(result_numerator == 0 or result_denumerator ==0):
		return 0.0
	return float(result_numerator) / float(result_denumerator)

def sentence_probability(gram_type,sentence,smooth):
	result = 0.0
	if(gram_type == "Unigram"):
		for word in sentence :
			if(word != SENTENCE_START and word != SENTENCE_END):
				if(result == 0.0):
					result = probability_unigram(word,smooth)
				else:
					result *= probability_unigram(word,smooth)

	elif(gram_type == "Bigram"):
		previous_word = None
		for word in sentence:
			if(previous_word != None  and word != SENTENCE_END):
				if(result == 0.0):
					result = probability_bigram(previous_word,word,smooth)
				else:
					result *= probability_bigram(previous_word,word,smooth)
			previous_word = word

	elif(gram_type =="Trigram"):
		first_word = None
		second_word = None
		for word in sentence:
			if(first_word != None and word != SENTENCE_END):
				if(result == 0.0):
					result = probability_trigram(first_word,second_word,word,smooth)
				else:
					result *= probability_trigram(first_word,second_word,word,smooth)
			previous_word = word
	return result




In [ ]:
train_sentences = "She is Beautiful. She is Wonderful. He is Greek."
sentences = preprocess_sentences(train_sentences)
preprocess(sentences)
input_sentences = preprocess_sentence("She is Wonderful")
sentence_probability("Bigram",input_sentences,False)

0.2222222222222222

In [ ]:
# [5%] Add an option to your program to compute the perplexity of a test set

def calculate_perplexity(gram_type,s,smooth):
	result = 0.0
	if(gram_type == "Unigram"):
		number_of_unigram = calcgram(s)
		temp = sentence_probability("Unigram",s,smooth)
		if(temp != 0.0):
			return math.pow(temp,-1 / number_of_unigram)
		else:
			return 0.0
		

	elif(gram_type == "Bigram"):
		number_of_bigram = calcgram(s)
		temp = sentence_probability("Bigram",s,smooth)
		if(temp != 0.0):
			return math.pow(temp,-1 / number_of_bigram)
		else:
			return 0.0		

	elif(gram_type =="Trigram"):
		number_of_trigram = calcgram(s)
		temp = sentence_probability("Trigram",s,smooth)
		if(temp != 0.0):
			return math.pow(temp,-1 / number_of_trigram)
		else:
			return 0.0
		
	return result

calculate_perplexity("Bigram",input_sentences,False)

1.6509636244473134

In [ ]:
# [10%] Add an option to your program to do smoothing or discounting

# Answer : To do smoothing, just put the False value into True (since the last parameter is for smoothing)
# Here in my source code, I only apply the add-1 smoothing (using this reference https://web.stanford.edu/~jurafsky/slp3/3.pdf)

input_sentences = preprocess_sentence("She is Gorgeous")
# sentence_probability("Bigram",input_sentences,False) # will get 0.0 as the result
sentence_probability("Bigram",input_sentences,True)

0.1875

In [ ]:
# [5%] Calculate the probability and the perplexity of a test set

newTrainSet = "Hope is the thing with feathers. That perches in the soul, And sings the tune without the words, And never stops at all, And sweetest in the gale is heard; And sore must be the storm. That could abash the little bird. That kept so many warm. I’ve heard it in the chillest land, And on the strangest sea; Yet, never, in extremity,It asked a crumb of me."
sentences = preprocess_sentences(newTrainSet)
preprocess(sentences)
input_sentences = preprocess_sentence("Hope is the thing with soul")
print("Probability without Smoothing = ", str(sentence_probability("Bigram",input_sentences,False)))
print("Probability with Smoothing = ", str(sentence_probability("Bigram",input_sentences,True)))
print("Perplexity without Smoothing = ", str(calculate_perplexity("Bigram",input_sentences,False)))
print("Perplexity with Smoothing = ", str(calculate_perplexity("Bigram",input_sentences,True)))


Probability without Smoothing =  0.0
Probability with Smoothing =  0.007407407407407408
Perplexity without Smoothing =  0.0
Perplexity with Smoothing =  2.2649344008227015


In [ ]:
# [5%] Compare the smoothed bigram and the smoothed trigram model of a test set.

print("Probability of Smoothed Bigram = ", str(sentence_probability("Bigram",input_sentences,True)))
print("Probability of Smoothed Trigram = ", str(sentence_probability("Trigram",input_sentences,True)))

# The Smoothed Bigram probability is greater than the Smoothed Trigram Probability.

Probability of Smoothed Bigram =  0.007407407407407408
Probability of Smoothed Trigram =  0.0
